In [1]:
import torch
import torch.nn as nn
import os
import sys
import open3d as o3d
from pathlib import Path

# addpath('../')
sys.path.append(os.path.join(Path().resolve(), '../')) # add parent directory to path
import ptlk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
print(torch.cuda.is_available())

True


In [3]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, transform = None):
        self.transform = transform

        self.data = [1, 2, 3, 4, 5, 6]
        self.label = [0, 1, 0, 1, 0, 1]

        self.datanum = 6

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label  

In [7]:
load_path = './model/ex1_pointlk_1030_model_best.pth'
load_weights = torch.load(load_path)
print(load_weights.keys())

odict_keys(['dt', 'ptnet.h1.0.weight', 'ptnet.h1.0.bias', 'ptnet.h1.1.weight', 'ptnet.h1.1.bias', 'ptnet.h1.1.running_mean', 'ptnet.h1.1.running_var', 'ptnet.h1.1.num_batches_tracked', 'ptnet.h1.3.weight', 'ptnet.h1.3.bias', 'ptnet.h1.4.weight', 'ptnet.h1.4.bias', 'ptnet.h1.4.running_mean', 'ptnet.h1.4.running_var', 'ptnet.h1.4.num_batches_tracked', 'ptnet.h2.0.weight', 'ptnet.h2.0.bias', 'ptnet.h2.1.weight', 'ptnet.h2.1.bias', 'ptnet.h2.1.running_mean', 'ptnet.h2.1.running_var', 'ptnet.h2.1.num_batches_tracked', 'ptnet.h2.3.weight', 'ptnet.h2.3.bias', 'ptnet.h2.4.weight', 'ptnet.h2.4.bias', 'ptnet.h2.4.running_mean', 'ptnet.h2.4.running_var', 'ptnet.h2.4.num_batches_tracked', 'ptnet.h2.6.weight', 'ptnet.h2.6.bias', 'ptnet.h2.7.weight', 'ptnet.h2.7.bias', 'ptnet.h2.7.running_mean', 'ptnet.h2.7.running_var', 'ptnet.h2.7.num_batches_tracked'])


In [9]:
load_weights['ptnet.h2.7.weight']

tensor([1.0894, 1.0228, 1.0139,  ..., 1.0908, 0.9976, 0.9806])

In [10]:
ptnet = ptlk.pointnet.PointNet_features()
base_model = ptlk.pointlk.PointLK(ptnet=ptnet)

In [13]:
# 事前学習済みの重みを読み込む
load_path = './model/ex1_pointlk_1030_model_best.pth'
load_weights = torch.load(load_path)
base_model.load_state_dict(load_weights)

<All keys matched successfully>

In [14]:
import torchvision
import torchvision.transforms as transforms

testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
)

0.5%


KeyboardInterrupt: 

In [ ]:
# dataloader
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1, shuffle=False)

# カスタムヘッドの追加
custom_head = nn.Sequential(
    nn.Linear(in_features, out_features),  # カスタムヘッドのアーキテクチャを設計
    nn.ReLU(),
    nn.Linear(out_features, num_classes)  # 分類の数に合わせた出力層
)

# 新しいモデルを構築
model = nn.Sequential(
    base_model,
    custom_head
)

learning_rate = 1e-3

# カスタムヘッドのパラメータのみをトレーニング
optimizer = torch.optim.SGD(custom_head.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

# データセットを使用してモデルをトレーニング
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # データをモデルに渡す
        scores = model(data)
        loss = criterion(scores, targets)

        # バックプロパゲーションとパラメータの更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# モデルを保存
torch.save(model.state_dict(), 'my_model.pth')